# Getting Started with Brightway2

This is an introduction to Brightway2, an open source framework for Life Cycle Assessment. This notebook will cover the basics of databases and activities and looking at LCI databases and LCIA methods.

At the end of this notebook, you will be able to:

* Import basic data like the biosphere database
* Import and explore the [FORWAST](http://forwast.brgm.fr/) database

If you finish the notebook, you get a kitten.

This introduction is written in an Jupyter notebook, an online scientific notebook which combines, text, data, images, and programming. It is amazing, and could be a fantastic way to do and communicate advanced LCA work. See the [documentation](http://ipython.org/ipython-doc/dev/interactive/htmlnotebook.html) and a list of [awesome examples](https://github.com/ipython/ipython/wiki/A-gallery-of-interesting-IPython-Notebooks). Please see the [Brightway2 documentation](https://docs.brightwaylca.org) for a complete list of [example Brightway2 notebooks](https://docs.brightwaylca.org).

You should **download this notebook** and run it cell by cell - don't just read it on the web!

# Starting at the beginning

Import brightway2.

In [1]:
from brightway2 import *

In [ ]:
projects.delete_project("BW2 introduction", True)

## Python 2 and 3

Brightway2 is compatible with both Python 2 and 3. If you are using Python 2, I strongly recommend you execute the following cell, which will make all your text string unicode without have to type the letter ``"u"`` in front of each text string. On Python 3, the following **doesn't do anything** - your text strings are unicode by default.

I won't include this cell in future notebooks, but you should copy and paste it if you will use Python 2.

In [2]:
from __future__ import unicode_literals, print_function

## Projects

In Brightway2, a project is a separate directory with its own copies of LCI databases, LCIA methods, and any other data you use. Each research project or article should probably be its own project, so that any changes you want to make will not interfere with your other work.

The default project is called ``default``:

In [3]:
projects.current

'default'

Each project is stored in a separate directory in a place in your filesystem reserved for application data. It varies depending on the operating system; on OS X, this is the `Library` directory in your user home directory:

In [4]:
projects.dir

'/Users/cmutel/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682'

However, you really need to care about the specifics. 

We can create a new project:

In [5]:
projects.set_current("BW2 introduction")

And list the available projects:

In [6]:
projects

Brightway2 projects manager with 33 objects, including:
	1
	3.2
	7
	8
	BW2 Tutorial 1
	BW2 Tutorial 4
	BW2 introduction
	CAES
	Class 2
	Class 3
Use `sorted(projects)` to get full list, `projects.report()` to get
	a report on all projects.

## Getting basic data

Let's import some basic data - a database of elementary flows, some LCIA methods, and some metadata used for importing other databases:

In [7]:
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:
0%                          100%
[####                          ] | ETA: 00:00:01

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/15/2016 09:30:12
  Finished: 07/15/2016 09:30:13
  Total time elapsed: 00:00:00
  CPU %: 93.60
  Memory %: 1.64
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 2.24 seconds
Wrote 665 LCIA methods with 169551 characterization factors
Creating core data migrations



The iPython notebook by default prints all logged messages. On your machine, there might be messages tracking how long it took to download and process the biosphere and methods packages.

## A biosphere dataset

The ``biosphere3`` database was installed. It is called the ``biosphere3`` database because elementary flow names are normalized to the ecoinvent 3 standard.

Let's see how many flows there are in this database, and then look at one random flow:

In [8]:
db = Database("biosphere3")
print("Number of flows in `biosphere3`:", len(db))
random_flow = db.random()
print(random_flow)

Number of flows in `biosphere3`: 4018
'Phthalate, dimethyl-' (kilogram, None, ('water',))


In [9]:
print(random_flow['name'])
print(random_flow['unit'])
print(random_flow['categories'])

Phthalate, dimethyl-
kilogram
('water',)


Brightway2 uses keys to identify datasets. Each dataset is identified by a combination of its database and some unique code. The code can be anything - a number, a UUID, or just a name. All of the following would be valid keys:

    ("biosphere", "f66d00944691d54d6b072310b6f9de37")
    ("my new database", "building my dream house")
    ("skynet", 14832)

In [10]:
random_flow.key

('biosphere3', '5e3a522a-d959-4aac-9ff0-c1c09f4e645e')

## An LCIA method dataset

We also installed a large number of LCIA methods:

In [11]:
len(methods)

665

Because LCIA methods have many different impact categories, they are identified not by a single label, but by a list of labels. Let's look at an example:

In [12]:
method_key = methods.random()
method_key

('ReCiPe Midpoint (I)', 'ionising radiation', 'IRP_I')

In this case, the LCIA method has three levels of specificity, from the general name (first level) to the specific impact category (last level). There is nothing magic about three levels - you could have one, or one hundred - but Brightway2 expects that LCIA methods will be a list of text labels ``('like', 'this')``.

Note that method identifiers need to be a special kind of list that uses `()` instead of `[]`. These are called `tuples`. To create a tuple with only one element, you need to add a comma, to distinguish it from a set of parentheses:

In [13]:
print (1 + 2)           # This is not a tuple
print (1,), type((1,))  # This is a tuple with one element

3
1


(None, tuple)

We can load the method data, show a sample.

Method data has the format:

    biosphere flow, numeric value, location

Where:

* `biosphere flow` is a dataset from any database which is used as a biosphere flow.
* `numeric value` can be either a fixed number or an uncertainty distribution.
* `location` is optional; the default value is that this characterization factor is valid everywhere.

The method data format is pretty flexible, and the following are all acceptable:

    [('biosphere', 'CO2'), 1.0],                                             # Default location
    [('biosphere', 'CO2'), 1.0, 'Australia, mate!'],                         # Custom location
    [('biosphere', 'CO2'), 1.0, ('Population density', 'Raster cell 4,2')],  # Location inside a geocollection
    [('biosphere', 'CO2'), {'amount': 1.0, 'uncertainty type': 0}],          # Uncertain characterization factor

[Geocollections](http://brightway2-regional.readthedocs.org/#spatial-scales-geocollections) are needed for regionalized LCA.

If you are wondering why we need to identify biosphere flows like `('biosphere', '2fe885840cebfcc7d56b607b0acd9359')`, this is a good question! The short answer is that there is no single field that uniquely identifies biosphere flows or activities. The longer answer [is in the manual](http://brightway2.readthedocs.org/en/latest/lci.html#uniquely-identifying-datasets).

Brightway2 is designed to be flexible enough for many different problems. Therefore, there are no limits on what constitutes a biosphere flow. Rather, anything that is linked to in a biosphere exchange will be put in the biosphere matrix. We installed a database called `biosphere3`, but you can define new flows in a database alongside process datasets, or create your own biosphere database.

In [14]:
method_data = Method(method_key).load()
print("Number of CFs:", len(method_data))
method_data[:20]

Number of CFs: 200


[(('biosphere3', 'f06815cb-65ca-4446-8ea9-329ae0252c29'), 2.51),
 (('biosphere3', 'c56df4f7-c9e9-4892-a045-4b2fb93ff297'), 2.51),
 (('biosphere3', '02703daf-fde4-489a-b7e5-523e46151312'), 2.51),
 (('biosphere3', 'd3805249-4c8b-48fb-8843-a7765b8d1322'), 2.51),
 (('biosphere3', 'e73fc455-cc04-4328-b961-1e388410e6c5'), 2.51),
 (('biosphere3', '58a92690-aa80-4d3c-9443-9b48daa2daa4'), 1.5),
 (('biosphere3', 'f9055607-c571-4903-85a8-7c20e3790c43'), 0.038571),
 (('biosphere3', '0e066010-5511-4ce5-b7ff-446e58417a65'), 0.038571),
 (('biosphere3', '077b92b0-0610-48ba-9ff8-55c13e98364b'), 0.038571),
 (('biosphere3', '64f81516-52e1-4ce6-a22c-84d3f7cc9a53'), 0.038571),
 (('biosphere3', '6a05e007-ce7e-4dd3-8cb7-54585031b8b5'), 0.0007),
 (('biosphere3', '284a896f-2aca-496e-9ab2-845525fbbc53'), 0.92857),
 (('biosphere3', '6e7276bb-64f5-4421-8696-b0103b687ddd'), 0.92857),
 (('biosphere3', '19256c0f-bea8-4e61-9499-8cc2f84953f8'), 0.92857),
 (('biosphere3', 'bfc66f26-0b11-4e6e-9e8c-2bfde24dc23c'), 0.9285

# Importing the FORWAST LCI database

We will use the FORWAST database, as it is both a high quality, comprehensive LCI database, and freely available. [FORWAST](http://forwast.brgm.fr/Overview.asp) is a physical MRIO table for Europe. It can be downloaded directly from the [2.-0 website](http://lca-net.com/projects/show/forwast/).

The following cell will download and install the FORWAST database. Note that an internet connection is required.

In [15]:
import zipfile
import os
from bw2data.utils import download_file

filepath = download_file("forwast.bw2package.zip", url="http://lca-net.com/wp-content/uploads/")
dirpath = os.path.dirname(filepath)
zipfile.ZipFile(filepath).extractall(dirpath)
BW2Package.import_file(os.path.join(dirpath, "forwast.bw2package"))

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:04


Title: Writing activities to SQLite3 database:
  Started: 07/15/2016 09:31:47
  Finished: 07/15/2016 09:31:52
  Total time elapsed: 00:00:04
  CPU %: 85.50
  Memory %: 3.42


[Brightway2 SQLiteBackend: forwast]

# Searching datasets

By default, every database is added to a search engine powered by [Whoosh](https://bitbucket.org/mchaput/whoosh/wiki/Home). Searching covers the following data fields:
* `name`
* `comment`
* `product`
* `categories`
* `location`

Searching is done by using the `Database.search` method.

In [16]:
Database("forwast").search("food")

['_26 Food preparations n.e.c., DK' (kilogram, GLO, ['Input Output', 'Denmark 2003']),
 '_26 Food preparations n.e.c., EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']),
 '101 Waste treatment, Incineration of waste, Food, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']),
 '116 Waste treatment, Landfill of waste, Food, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']),
 '112 Waste treatment, Composting of food waste, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']),
 '_92 Waste treatment, Incineration of waste, Food, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003']),
 '103 Waste treatment, Composting of food waste, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003']),
 '100 Waste treatment, Biogasification of food waste, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003']),
 '107 Waste treatment, Landfill of waste, Food, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003']),
 '109 Waste treatment, Biogasification of food waste, EU27' (kilogram, GLO, ['Input Output', 

Searches can also be filtered (where only the results that meet the specified criteria are *included*) or masked (where results that meet the specified criteria are *excluded*).

By default we return 25 search results, but this can be changed by specifying the `limit` keyword argument.

You can also use `*` wild cards in search queries:

In [17]:
Database("biosphere3").search("carb*", limit=10)

['Carbon-14' (kilo Becquerel, None, ('water', 'surface water')),
 'Carbon disulfide' (kilogram, None, ('water', 'surface water')),
 'TOC, Total Organic Carbon' (kilogram, None, ('water', 'ground-, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Hydrogen carbonate' (kilogram, None, ('water', 'ocean')),
 'Carbon monoxide, from soil or biomass stock' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'industrial')),
 'Carbon monoxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Elemental carbon' (kilogram, None, ('soil',)),
 'Carbon' (kilogram, None, ('soil', 'agricultural'))]

You can specify inclusion or exclusion criteria for fields with `filter` and `mask`:

In [18]:
Database("biosphere3").search("carbon", filter={"categories": 'forestry'})

Excluding 77 filtered results


['Carbon' (kilogram, None, ('soil', 'forestry')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'forestry'))]

In [19]:
Database("biosphere3").search("carbon", limit=10, mask={"categories": 'forestry'})

Excluding 2 filtered results


['Carbon' (kilogram, None, ('soil', 'agricultural')),
 'Carbon' (kilogram, None, ('soil', 'industrial')),
 'Carbon' (kilogram, None, ('soil',)),
 'Carbon-14' (kilo Becquerel, None, ('water', 'surface water')),
 'Carbon disulfide' (kilogram, None, ('water', 'surface water')),
 'Elemental carbon' (kilogram, None, ('soil',)),
 'Carbon-14' (kilo Becquerel, None, ('water', 'ocean')),
 'Carbon disulfide' (kilogram, None, ('water', 'ocean')),
 'Organic carbon' (kilogram, None, ('air', 'urban air close to ground')),
 'Organic carbon' (kilogram, None, ('soil',))]

Finally, you can facet search results by another field. This is a bit complicated, so test your queries before assuming certain behaviour.

In [20]:
sr = Database("biosphere3").search("carbon", facet="categories")
for key, value in sr.items():
    print("Facet term:", key, "\n\t", value[:3], "\n")

Facet term: air 
	 ['Carbon disulfide' (kilogram, None, ('air',)), 'Carbon-14' (kilo Becquerel, None, ('air',)), 'Carbon dioxide, fossil' (kilogram, None, ('air',))] 

Facet term: term 
	 ['Carbon disulfide' (kilogram, None, ('air', 'low population density, long-term')), 'Carbon-14' (kilo Becquerel, None, ('air', 'low population density, long-term')), 'Carbon monoxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))] 

Facet term: soil 
	 ['Carbon' (kilogram, None, ('soil', 'agricultural')), 'Carbon' (kilogram, None, ('soil', 'forestry')), 'Carbon' (kilogram, None, ('soil', 'industrial'))] 

Facet term: indoor 
	 ['Carbon dioxide, from soil or biomass stock' (kilogram, None, ('air', 'indoor')), 'Carbon monoxide, from soil or biomass stock' (kilogram, None, ('air', 'indoor'))] 

Facet term: upper 
	 ['Carbon-14' (kilo Becquerel, None, ('air', 'lower stratosphere + upper troposphere')), 'Carbon disulfide' (kilogram, None, ('air', 'lower stratosphere + upper troposp

## Changing iteration order

You can also change the way that processes are iterated over in the database. The default order is random:

In [21]:
db = Database("forwast")

def print_10(db):
    for i, x in enumerate(db):
        if i < 10:
            print(x)
        else:
            break
            
print_10(db)

'_55 Aluminium basic, virgin, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_33 Pulp, virgin, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_11 Crude petroleum and natural gas, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_63 Transport equipment n.e.c., DK' (EUR2003, GLO, ['Input Output', 'Denmark 2003'])
'__6 Agricultural services n.e.c., EU27' (EUR2003, GLO, ['Input Output', 'EU27 2003'])
'_48 Recycling of concrete, asphalt and other mineral products, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'_31 Leather products, footwear, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'133 Household use, Social care, DK' (EUR2003, GLO, ['Input Output', 'Denmark 2003'])
'104 Waste treatment, Composting of paper and wood, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'114 Waste treatment, Landfill of waste, Glass' (kilogram, inert, DK, ['Input Output', 'Denmark 2003'])


In [22]:
print_10(db)

'dK total production and consumption 2003' (unit, GLO, ['Input Output', 'Denmark 2003\\Final use'])
'_64 Furniture and other manufactured goods n.e.c., DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'_62 Aluminium, after first processing, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_14 Copper from mine, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_94 Waste treatment, Incineration of waste, Plastic, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'_77 Water, fresh, EU27' (EUR2003, GLO, ['Input Output', 'EU27 2003'])
'104 Waste treatment, Composting of paper and wood, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'_75 Hotels and restaurants, DK' (EUR2003, GLO, ['Input Output', 'Denmark 2003'])
'_54 Recycling of iron basic, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'_15 Metals from mine n.e.c., EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])


You can sort by `location`, `name`, `product` (reference product), or `type`, by specifying the `order_by` property.

In [23]:
db.order_by = "name"
print_10(db)

'100 Health and social work, EU27' (EUR2003, GLO, ['Input Output', 'EU27 2003'])
'100 Waste treatment, Biogasification of food waste, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'101 Waste treatment, Biogasification of paper, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'101 Waste treatment, Incineration of waste, Food, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'102 Waste treatment, Biogasification of sewage slugde, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'102 Waste treatment, Incineration of waste, Paper, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'103 Waste treatment, Composting of food waste, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'103 Waste treatment, Incineration of waste, Plastic, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])
'104 Waste treatment, Composting of paper and wood, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])
'104 Waste treatment, Incineration of waste, Metals, EU27' (kilogram, GLO, [

If the above seems wrong, remember that the names start with `100`, `101`, etc.

Set `.order_by = None` to remove any ordering.

In [24]:
db.order_by = None

Because accessing activities in the database is quite fast, you can also filter the activities you want by just iterating over the entire database:

In [25]:
my_activities = [x for x in db if 'Electr' in x['name']]
my_activities

['_66 Electricity, steam and hot water, DK' (kilowatt hour, GLO, ['Input Output', 'Denmark 2003']),
 '_75 Electricity, steam and hot water, EU27' (kilowatt hour, GLO, ['Input Output', 'EU27 2003']),
 '_68 Electrical machinery n.e.c., EU27' (kilogram, GLO, ['Input Output', 'EU27 2003']),
 '_59 Electrical machinery n.e.c., DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])]

# Basic LCA calculations

Let's pick and random process and LCIA method:

In [26]:
process = Database("forwast").random()
process

'_69 Radio, television and communication equipment, EU27' (kilogram, GLO, ['Input Output', 'EU27 2003'])

A brief review of LCA calculations:

In matrix-based LCA, we construct a *technosphere* matrix, which describes the inputs needed to produce different products (e.g. cars need metal and electricity), and a *biosphere* matrix, which describes the emissions and resource consumption associated with the production of each product (e.g. car manufacturing releases air emissions). These two matrices come from the life cycle inventory database(s). We also have a *functional unit*, which is what we are trying to assess, e.g. one car. We then calculate the *life cycle inventory* (LCI) by first solving the linear system of the technosphere matrix and the functional unit, and then by multiplying the biosphere matrix.

To do *life cycle impact assessment* (LCIA), we multiply the life cycle inventory by a matrix of characterization factors, which tell how bad different emissions and resource consumptions are.

For more details on the math, see the [manual](https://docs.brightwaylca.org/lca.html).

So, our first step is to specify the functional unit, which is relatively easy:

In [27]:
functional_unit = {process: 1}

We can then instantiate our [LCA object](https://docs.brightwaylca.org/technical/bw2calc.html#lca). 

In [28]:
lca = LCA(functional_unit, method_key)

And do the LCI and LCIA calculations:

In [29]:
lca.lci()
lca.lcia()

Finally, we can print the LCA score:

In [30]:
lca.score

0.0

You can reuse the same matrices but change the functional unit by using the `redo_lci` and `redo_lcia` functions:

In [31]:
new_process = Database("forwast").random()
print(new_process)
lca.redo_lcia({new_process: 1})
lca.score

'_55 Iron, after first processing, DK' (kilogram, GLO, ['Input Output', 'Denmark 2003'])


0.0

# Looking into the LCA object

Let's see what is in this `LCA` thing, anyway. Put your cursor in the following cell and hit tab:

In [32]:
lca.

SyntaxError: invalid syntax (<ipython-input-32-0c6d38382cd8>, line 1)

So, there is a lot. Let's look at a few things:

* The technosphere matrix

In [ ]:
lca.technosphere_matrix

* The biosphere matrix

In [ ]:
lca.biosphere_matrix

* The characterization matrix

In [ ]:
lca.characterization_matrix

# Graphing matrices

Sometimes it can be helpful to visualize both the inputs and calculation results.

In [ ]:
%matplotlib inline

In [ ]:
from bw2analyzer.matrix_grapher import SparseMatrixGrapher

First, let's look at the technosphere matrix.

You can pass in a filename to get a higher resolution figure saved as a file.

In [ ]:
SparseMatrixGrapher(lca.technosphere_matrix).ordered_graph()

Not so interesting - I am sure your inventory data will be much nicer. The problem is that this is an IO matrix, so there is a value at each point, even if it is small, and this graph only shows where values are or aren't zero. For example, here is the same graph for ecoinvent 3.2 (cutoff):

<img src="https://brightwaylca.org/data/ecoinvent-32-cutoff.png">

You can also graph the biosphere and any other LCA matrices.

# Contribution analysis

We can calculate the most important activities and biosphere flows.

In [ ]:
from bw2analyzer import ContributionAnalysis

Most important activities

In [ ]:
ContributionAnalysis().annotated_top_processes(lca)

Most important biosphere flows

In [ ]:
ContributionAnalysis().annotated_top_emissions(lca)

# Monte Carlo LCA

Unfortunately, the forwast database doesn't unclude uncertainty. Let's put some in anyways, using the utility function [uncertainify](https://docs.brightwaylca.org/technical/bw2data.html#bw2data.utils.uncertainify).

In [ ]:
from bw2data.utils import uncertainify
from stats_arrays import NormalUncertainty

In [ ]:
uncertain_db = Database("forwast uncertain +")
uncertain_db.write(
    uncertain_db.relabel_data(
        uncertainify(
            Database("forwast").load(), 
            NormalUncertainty
        ), 
        "forwast uncertain +" 
    )
)

We can now calculate some Monte Carlo iterations for a random activity.

In [ ]:
mc = MonteCarloLCA(demand={uncertain_db.random(): 1}, method=method_key)
mc.load_data()
for x in range(10):
    print(next(mc))

That't it! Here is your promised kitten:

In [ ]:
from IPython.display import Image
import random
dimensions = sorted((int(random.random() * 600 + 200), int(random.random() * 600 + 200)))
Image(url="http://placekitten.com/{}/{}/".format(*dimensions))